In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from tensorflow.keras.callbacks import LambdaCallback
import os

In [61]:
tezte = pd.read_csv('/home/lar-desktop-ur5/Área de Trabalho/danielufba/icdaniel/mitbih_train.csv')

KeyboardInterrupt: 

In [ ]:
def divide_em_segments(dataset, tamanho_segmento): # Função para dividir o dataset em segmentos com um número fixo de colunas (tamanho_segmento).

    segmentos = []
    distancia = dataset.shape[1] - tamanho_segmento + 1

    for i in range(dataset.shape[0]):
        for j in range(distancia):
            segmentos.append(dataset.iloc[i, j:(j + tamanho_segmento)].values)

    dataset_segmentado = pd.DataFrame(segmentos)  # Criar o DataFrame com os segmentos
    return dataset_segmentado

In [ ]:
def preenche_zeros_com_valor(dataset): # Função para substituir os zeros nas últimas posições de cada linha com NaN.
    for i in range(dataset.shape[0]):
        for j in range(dataset.shape[1]-1):
            if dataset.iloc[i, -(j+1)] == 0:
                dataset.iloc[i, -(j+1)] = np.nan
            else:
                break
    return dataset

In [ ]:
def preprocessamento(dataset, tamanho_segmento):
    dataset = preenche_zeros_com_valor(dataset)
    dataset = divide_em_segments(dataset, tamanho_segmento)
    dataset = dataset.dropna()

    return dataset

In [ ]:
tezte.shape

In [ ]:
tezte.head()

In [ ]:
novos_nomes = [f'col{i+1}' for i in range(tezte.shape[1])]
novos_nomes[-1] = 'target'
tezte.columns = novos_nomes
tezte.describe()

In [ ]:
dataset_n = tezte[tezte['target'] == 0]
dataset_n = dataset_n.drop('target', axis=1)
datasetnormal, datasetnormal2 = train_test_split(dataset_n, test_size=0.5, random_state=42)

In [ ]:
datasetnormal.shape

In [ ]:
log_file = "training_log.txt"

# Apagar o conteúdo do arquivo e adicionar uma nova mensagem
with open(log_file, "w") as log:
    log.write("Iniciou o Processo 2 2\n")

# Enviar a mensagem para o tmux
os.system(f'tmux send-keys -t danieltrain "echo \\"Iniciou o Processo\\"" C-m')


In [ ]:
dataset_segmentando = preprocessamento(datasetnormal, 10)

In [ ]:
log_file = "training_log.txt"
with open(log_file, "a") as log:
    log.write("Terminou de fazer o dataset\n")

os.system(f'tmux send-keys -t danieltrain "echo \\"Terminou o Dataset\\"" C-m')


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataset_segmentando_scaled = scaler.fit_transform(dataset_segmentando)
dataset_segmentando_scaled = pd.DataFrame(dataset_segmentando_scaled, columns=dataset_segmentando.columns)

In [ ]:
x = dataset_segmentando_scaled.iloc[:, :-1]
y = dataset_segmentando_scaled.iloc[:, -1]
xsegmentando = np.array(x)
ysegmentando = np.array(y)
xsegmentando = xsegmentando.reshape((xsegmentando.shape[0], xsegmentando.shape[1], 1))
ysegmentando = ysegmentando.reshape(-1, 1)  

In [ ]:
print (xsegmentando.shape)
print (ysegmentando.shape)

In [75]:
import os
from datetime import datetime
from tensorflow.keras.callbacks import LambdaCallback,ModelCheckpoint, EarlyStopping



# Nome do arquivo de log
log_file = "training_log.txt"

# Callback para salvar no início do treinamento e no início de cada época
training_start_callback = LambdaCallback(
    on_train_begin=lambda logs: (
        open(log_file, "a").write(f" Treinamento iniciado!\n"),
        os.system(f'tmux send-keys -t danieltrain "Treinamento iniciado!" C-m')
    ),
    on_epoch_begin=lambda epoch, logs: (
        open(log_file, "a").write(f" Início da Epoch {epoch+1} {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"),
        os.system(f'tmux send-keys -t danieltrain " Início da Epoch {epoch+1} " C-m')
    )
)

# Callback para salvar no final de cada época
tmux_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: (
        # Salvar no arquivo de log
        open(log_file, "a").write(
            f" Epoch {epoch+1} - Loss: {logs['loss']:.4f} - Val Loss: {logs['val_loss']:.4f}\n"
        ),
        # Enviar para o tmux
        os.system(
            f'tmux send-keys -t danieltrain " Epoch {epoch+1} - Loss: {logs["loss"]:.4f} - Val Loss: {logs["val_loss"]:.4f}" C-m'
        )
    )
)

# Callback para salvar o melhor modelo
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint6.h5', 
    save_best_only=False,            
    save_weights_only=False,        
    verbose=1                        
)

# Callback para parar o treinamento cedo se não houver melhora
early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,           
    restore_best_weights=True  
)



In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
import matplotlib.pyplot as plt


In [77]:
from keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)



In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(xsegmentando.shape[1], xsegmentando.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(64))
model.add(Dropout(rate=0.1))
model.add(Dense(16))
model.compile(optimizer='adam', loss='mae')
model.summary()
history = model.fit(xsegmentando, ysegmentando, epochs=50, batch_size=32, validation_split=0.1, verbose=1, callbacks=[training_start_callback, tmux_callback, checkpoint_callback, early_stopping])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 9, 128)            66560     
                                                                 
 dropout_24 (Dropout)        (None, 9, 128)            0         
                                                                 
 lstm_25 (LSTM)              (None, 9, 128)            131584    
                                                                 
 dropout_25 (Dropout)        (None, 9, 128)            0         
                                                                 
 lstm_26 (LSTM)              (None, 9, 64)             49408     
                                                                 
 dropout_26 (Dropout)        (None, 9, 64)             0         
                                                                 
 lstm_27 (LSTM)              (None, 64)               

In [ ]:
with open(log_file, "a") as log:
    log.write("Treinamento Terminou\n")

os.system(f'tmux send-keys -t danieltrain "echo \\"Terminou o Treinamento\\"" C-m')

In [ ]:
prevteste = model.predict(xsegmentando)

In [1]:
erros = np.abs(prevteste - ysegmentando)

mae_vetores = np.mean(erros, axis=1)

mae_geral = np.mean(erros)

vetores_maiores = mae_vetores > mae_geral
quantidade_vetores_maiores = np.sum(vetores_maiores)
quantidade_vetores = len(mae_vetores)

print("MAE de cada vetor:", mae_vetores)
print("MAE geral:", mae_geral)
print("Quantidade de vetores com erro médio maior que o MAE geral:", quantidade_vetores_maiores)
print("Quantidade de vetores :", quantidade_vetores)


NameError: name 'np' is not defined

In [ ]:
plt.boxplot(mae_vetores)

In [ ]:
erros[0].sum()/len(erros[0])

In [ ]:
lista = []
for i in range(len(mae_vetores)):
    if mae_vetores[i]>mae_geral:
        lista.append(i)

In [ ]:
n = 68164
print("Preditor:",xsegmentando[n])
print("predição:",prevteste[n])
print("Valor Real:",ysegmentando[n])
print("Erro:",mae_vetores[n])

In [ ]:
indices_maiores_que_2 = np.where(mae_vetores > 2)[0]

print("Índices onde os valores são maiores que 2:", indices_maiores_que_2)